- La idea de este proyecto es subir un csv para limpiar.

In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None)  # Sin límite de filas

In [2]:
# pip install Jinja2

In [3]:
# df_ejemplo.style.highlight_null(color='yellow')

In [4]:
# cargamos el dataframe creado:
df_excel = pd.read_excel("C:/Users/isaw9/OneDrive/Escritorio/ADALAB/Modulo_3/files/New_clothesforyou.xlsx", index_col = None)
df_excel.head()

,id,producto,precio (€),descuento (%),color,pais_origen,temporada2025,ventas_unidad,inventario,marca,estacion
0,1,Camiseta Fresh Breeze,23.75,0.1,azul,ES,Yes,34,213,A,2
1,2,Polo Urban Fit,45.6,15,rojo,IT,No,15,45,B,2
2,3,Blusa Soft Elegance,78.5,20,NaN,FR,No,12,3,C,3
3,4,Camisa Denim Classic,12.3,5,verde,PR,Si,78,534,D,2
4,5,Crop Top Sunset Glow,41.4,5,nergo,AL,No,65,NaN,A,3


In [5]:
# Estructura de limpieza de datos:

# - Estudio de mis datos: Saber que datos son, a que corresponden, columnas...
# - Finalidad y preguntas del estudio.
# - Limpieza de datos adecuada a la finalidad, con criterio y justificaciones de cambios.

In [6]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             115 non-null    int64 
 1   producto       115 non-null    object
 2   precio (€)     110 non-null    object
 3   descuento (%)  110 non-null    object
 4   color          103 non-null    object
 5   pais_origen    107 non-null    object
 6   temporada2025  104 non-null    object
 7   ventas_unidad  109 non-null    object
 8   inventario     99 non-null     object
 9   marca          102 non-null    object
 10  estacion       95 non-null     object
dtypes: int64(1), object(10)
memory usage: 10.0+ KB


In [7]:
# Tengo un dataframe de una tienda de ropa, que recoge diferentes datos de cada prenda. 
# Tenemos estas columnas:

df_excel.columns

Index(['id', 'producto', 'precio (€)', 'descuento (%)', 'color', 'pais_origen',
       'temporada2025', 'ventas_unidad', 'inventario', 'marca', 'estacion'],
      dtype='object')

In [8]:
# todas de tipo objeto, por lo que las columnas numéricas no categóricas, convendrá convertirlas a int/float llegado el momento.

In [9]:
# Haciendo una valoraicón de calores únicos:

df_excel.nunique()

id               114
producto         100
precio (€)        73
descuento (%)     12
color             10
pais_origen        8
temporada2025      7
ventas_unidad     29
inventario        48
marca             13
estacion           9
dtype: int64

In [10]:
# OBJETIVOS DEL ESTUDIO

# 1.-Conocer el valor de ventas totales que se ha obtenido (sin descuento)
# 2.-Saber que artículos no son de temporada, y cuanto se obtuvo (€) por ellos
# 3.-Calcular porcentaje de prendas por estación ¿en qué estación se vende más?
# 4.-Artículos con la palabra cozy en el nombre
# 5.-Calcular nueva columna con el descuento aplicado, y valorar la diferencia de ganancias respecto al artículo sin descuento
# 6.-Mayor país proveedor, e ingresos por país, tenemos que disolver el contrato con el país con menores ingresos.
# 7.-¿Cuál es el color favorito de la temporada? ¿y del invierno?
# 8.-Marca con mayores ventas --> Para enviar detalle por parte de la empresa
# 9.-Hacer una relación de ventas e inventario, para eliminar 14 prendas de nuestro stock.
# 10.- Elaborar un informe final.

In [11]:
""" Para concoer mis datos, puedo hacer un bulce for o try o except buscando si mis columnas son int
y sino que me saque una lista de errores en el except con un .apend. REPASAR .map
"""

' Para concoer mis datos, puedo hacer un bulce for o try o except buscando si mis columnas son int\ny sino que me saque una lista de errores en el except con un .apend. REPASAR .map\n'

In [12]:
print(f"El número de filas que tenemos es {df_excel.shape[0]}, y el número de columnas es {df_excel.shape[1]}")

El número de filas que tenemos es 115, y el número de columnas es 11


In [13]:
# Conociendo los objetivos del estudio, lo primero que habrá que hacer es una limpieza de nuestros datos:

df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             115 non-null    int64 
 1   producto       115 non-null    object
 2   precio (€)     110 non-null    object
 3   descuento (%)  110 non-null    object
 4   color          103 non-null    object
 5   pais_origen    107 non-null    object
 6   temporada2025  104 non-null    object
 7   ventas_unidad  109 non-null    object
 8   inventario     99 non-null     object
 9   marca          102 non-null    object
 10  estacion       95 non-null     object
dtypes: int64(1), object(10)
memory usage: 10.0+ KB


In [14]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df_excel[df_excel.columns[df_excel.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['precio (€)', 'descuento (%)', 'color', 'pais_origen', 'temporada2025',
       'ventas_unidad', 'inventario', 'marca', 'estacion'],
      dtype='object')


In [15]:
df_excel.describe(include = "object").T

,count,unique,top,freq
producto,115,100,Polo Urban Fit,2
precio (€),110.0,73.0,15.2,5.0
descuento (%),110,12,5,23
color,103,10,azul,19
pais_origen,107,8,ES,30
temporada2025,104,7,No,36
ventas_unidad,109,29,21,7
inventario,99,48,45,7
marca,102,13,D,30
estacion,95,9,1,28


In [16]:
# como hemos visto lo anterior nos devuelve una serie de booleanos lo cual es poco entendible, 
# por lo que añadiremos el método '.sum()' para que nos cuente cuantas filas tenemos duplicadas
# en este caso el código nos esta diciendo que no tenemos filas duplicadas

df_excel.duplicated().sum() # No tenemos valores duplicados(PERO OJO EL id 2 está repetido.)

np.int64(0)

In [17]:
# id: Serie de números enteros sin alteración. Tipo objeto --> convertir a int (aunque no necesario, es un identificador) 

In [18]:
# producto: Todo bien, un str.

df_excel["producto"].nunique() # cuenta el número de valores únicos en la columna "producto"

# Hay prendas con el mismo nombre, de momento no sabemos si datos repetidos, o quizás tienen diferentes características

100

In [19]:
#precio (€)

In [20]:
#Búsqueda de errores en una columna, por ejemplo la columna precio:

# Tipo: objetc, y la queremos como tipo float

#Vamos a cambiar las comas por puntos, pero es posible que nos encontremos más datos erróneos

errores_conversion = []  # Lista para almacenar los valores no convertidos

def cambiar_comas(valor):
    try:
        if isinstance(valor, str):  # Solo intenta reemplazar si es un string
            return float(valor.replace(",", "."))
        return valor  # Si ya es un número, lo deja igual
    except ValueError:  # Si la conversión falla, lo guarda en la lista
        errores_conversion.append(valor)
        return valor  # Devuelve el valor original si falla

In [21]:
df_excel["precio (€)"].apply(cambiar_comas)

0          23.75
1           45.6
2           78.5
3           12.3
4           41.4
5           67.8
6             20
7           90.1
8            NaN
9           67.4
10           3.1
11         14.37
12          15.2
13         11.12
14          20.9
15            30
16          45.6
17          74.3
18          20.3
19          20.3
20            25
21          11.5
22          12.4
23          27.5
24     33.133333
25     41.133333
26     49.133333
27     57.133333
28     65.133333
29     73.133333
30           NaN
31          41.4
32          67.8
33            20
34          90.1
35           NaN
36     54.689286
37           NaN
38     51.496429
39          49.9
40     48.303571
41     46.707143
42     45.110714
43           NaN
44     41.917857
45     40.321429
46        38.725
47         11.25
48         14.37
49          15.2
50         11.12
51        13.095
52        13.139
53        13.183
54         14.37
55          15.2
56         11.12
57     43.514286
58     41.9178

In [22]:
errores_conversion

['25 $']

In [23]:
# Ahora le digo que itere por mi lista errores de conversión, y me quite el simbolo del dolar.:
errores_corregidos =[]
for valor in errores_conversion:
    if isinstance(valor, str):  # Asegurar que es un string
        valor = valor.replace("$", "").replace(",", ".")  # Quitar $ y cambiar , por .
        
        try:
            errores_corregidos.append(float(valor))  # Intentar convertir a número
        except ValueError:
            errores_corregidos.append(valor)  # Si falla, guardar el valor original

print("Valores corregidos:", errores_corregidos)

Valores corregidos: [25.0]


In [24]:
# Vamos a limpiar la columna descuento: Le pasamos un filtro para quitar el % que no lo queremos, queremos datos int

In [25]:
for valor in df_excel["descuento (%)"]:
    print(f"Valor: {valor} -> Tipo: {type(valor)}")

Valor: 0.1 -> Tipo: <class 'float'>
Valor: 15 -> Tipo: <class 'int'>
Valor: 20 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: verde -> Tipo: <class 'str'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: nan -> Tipo: <class 'float'>
Valor: 15 -> Tipo: <class 'int'>
Valor: 20 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 10 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 10 -> Tipo: <class 'int'>
Valor: 15 -> Tipo: <class 'int'>
Valor: 20 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 10 -> Tipo: <class 'int'>
Valor: 15 -> Tipo: <class 'int'>
Valor: 20 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 10 -> Tipo: <class 'int'>
Valor: 15 -> Tipo: <class 'int'>
Valor: 20 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 10 -> Tipo: <class 'int'>
Valor: 15 -> Tipo: <class 'int'>
Valor: 20 -> Tipo: <class 'int'>
Valor: 5 -> Tipo: <class 'int'>
Valor: 10 -

In [26]:
errores_descuento = []

def quitar_porcentaje(valor):
    try:
        # Asegurarnos de que el valor es una cadena de texto
         # Convertimos el valor a string explícitamente
        if "%" in valor:
            # Eliminar el símbolo '%' y convertir el valor a int
            return (valor.replace("%", ""))
        return valor  # Si no tiene %, lo dejamos igual
    except ValueError:
          # Guarda los valores que no se pueden convertir
        return np.nan  # Devuelve el valor original si no se puede convertir




In [27]:
df_excel["descuento (%)"]= df_excel["descuento (%)"].apply(quitar_porcentaje)

TypeError: argument of type 'float' is not iterable

In [ ]:
df_excel

,id,producto,precio (€),descuento (%),color,pais_origen,temporada2025,ventas_unidad,inventario,marca,estacion
0,1,Camiseta Fresh Breeze,23.75,0.1,azul,ES,Yes,34,213,A,2
1,2,Polo Urban Fit,45.6,15,rojo,IT,No,15,45,B,2
2,3,Blusa Soft Elegance,78.5,20,NaN,FR,No,12,3,C,3
3,4,Camisa Denim Classic,12.3,5,verde,PR,Si,78,534,D,2
4,5,Crop Top Sunset Glow,41.4,5,nergo,AL,No,65,NaN,A,3
5,6,Sudadera Cozy Chill,67.8,verde,blanco,ES,Si,21,356,D,1
6,7,Jersey Warm Essence,20,5,azul,IT,0,52.6,245,A,1
7,8,Chaqueta Midnight Vibes,90.1,5,NaN,FR,0,56.914286,132,D,1
8,9,Blazer Power Look,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Chamarra Wild Road,67.4,15,verde,ES,Yes,65.542857,346,B,4


In [ ]:
errores_descuento #Nos encontramos que hay dos datos que "no tienen dato", podemos convertilos a nan, o hacer un relace por 0 sobrescribienod

['%', '%']

In [ ]:
df_excel["descuento (%)"] = df_excel["descuento (%)"].apply(quitar_porcentaje)

In [ ]:
df_excel["descuento (%)"]

0          10
1          15
2          20
3           5
4           5
5       verde
6           5
7           5
8         nan
9          15
10         20
11          5
12         10
13          5
14         10
15         15
16         20
17          5
18         10
19         15
20         20
21          5
22         10
23         15
24         20
25          5
26         10
27         15
28         20
29          5
30         10
31         15
32         20
33         20
34          5
35         10
36         15
37         20
38          5
39         10
40         15
41         20
42          5
43         10
44         15
45         20
46        0.2
47        0.1
48        0.1
49          5
50         10
51         15
52         20
53          0
54          0
55          0
56          5
57         10
58         15
59         20
60       0.05
61          5
62         10
63         15
64         20
65        0.1
66          5
67         10
68         15
69         20
70          0
71    

In [ ]:
df_excel["descuento (%)"] = df_excel["descuento (%)"].replace("verde","0").replace("quince","15")

In [ ]:
df_excel

,id,producto,precio (€),descuento (%),color,pais_origen,temporada2025,ventas_unidad,inventario,marca,estacion
0,1,Camiseta Fresh Breeze,23.75,10,azul,ES,Yes,34,213,A,2
1,2,Polo Urban Fit,45.6,15,rojo,IT,No,15,45,B,2
2,3,Blusa Soft Elegance,78.5,20,NaN,FR,No,12,3,C,3
3,4,Camisa Denim Classic,12.3,5,verde,PR,Si,78,534,D,2
4,5,Crop Top Sunset Glow,41.4,5,nergo,AL,No,65,NaN,A,3
5,6,Sudadera Cozy Chill,67.8,0,blanco,ES,Si,21,356,D,1
6,7,Jersey Warm Essence,20,5,azul,IT,0,52.6,245,A,1
7,8,Chaqueta Midnight Vibes,90.1,5,NaN,FR,0,56.914286,132,D,1
8,9,Blazer Power Look,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Chamarra Wild Road,67.4,15,verde,ES,Yes,65.542857,346,B,4
